# VRR Trend (50% vs 25% vol) Analysis
This notebok runs analysis on the 25% varying vol VRR Trend strategy.

## Import Libraries

In [ ]:
import os
os.chdir('..')

import plotly.express as px
import plotly.graph_objects as go
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
import pandas as pd

RUN_REPORTS = False

# Single Strats Analysis (VRR vs VRR2 vs VRR Trend - 50% vs VRR Trend - 25%)

## Import Equity & FI Benchmark Data

In [ ]:
equity_bmk = 'SPTR'
include_fi = True
strat_drop_list = ['99%/90% Put Spread','Down Var', 'Vortex','VOLA', 'Dynamic Put Spread', 'VRR',
       'GW Dispersion', 'Corr Hedge', 'Def Var']

#create returns data dictionary for equity benchmark
ret_dict= dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Import VRR, VRR2, VRR Trend Data and Display Historical Performance

In [ ]:
vrr = pd.read_excel(dm.NEW_DATA+'vrr_data.xlsx', sheet_name='data_25_pct', index_col=0)
fig = go.Figure()
for strat in vrr.columns:
    fig.add_trace(go.Scatter(x = pd.to_datetime(vrr.index, format = '%m/%d/%Y'),
                             y = vrr[strat],mode = 'lines',name = strat))
fig.update_layout(
    title={
        'text': "<b>Historical Performance</b>",
        'y':0.9,
        'x':0.5,'xanchor': 'center','yanchor': 'top'
    },
    title_font_family="Calibri",
    titlefont = {"size":20},
    xaxis = dict(tickfont = dict(size=14)),
    showlegend=True,
    plot_bgcolor='White'
    )
fig.update_xaxes(showline=True,linewidth=2,linecolor='black',mirror=False)
fig.update_yaxes(showline=True,linewidth=2,linecolor='black',mirror=False)
fig.show()

## Merge Data

In [ ]:
vrr_dict = dm.get_data_dict(vrr)
ret_data_dict_1 = dm.merge_dicts(ret_dict, vrr_dict)

# Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict_1 = summary.get_corr_data(ret_data_dict_1, include_fi=include_fi)

# Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **corr** - Show:
    * **full** - full history correaltions
    * **equity_down** - risk seeking benchmark downside returns correlations or 
    * **equity_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict_1[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict_1 = summary.get_analytics_data(ret_data_dict_1,include_fi=True)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_1[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict_1 = summary.get_hist_data(ret_data_dict_1)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict_1[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **bmk** - Equity or FI bmk

In [ ]:
group_list=['Quintile','Decile']
bmk_list = list(ret_data_dict_1['Monthly'].select_dtypes('float').columns)[0:2]
@interact
def display_returns(group = group_list, bmk = bmk_list):
    quintile_df = summary.get_grouped_data(ret_data_dict_1, group = group, strat = bmk)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list,bmk = bmk_list, strat=list(ret_data_dict_1['Monthly'].select_dtypes('float').columns)[2:]):    
    df = summary.get_grouped_data(ret_data_dict_1, group = group, strat=bmk)
    
    for col in df.columns:
        if not (col == bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    

# VRR + Trend Strategy Analysis

## Add VRR+Trend tracks and merge to Bmk

In [ ]:
vrr_trend = vrr.copy()
vrr_trend['VRR+Trend-50% (85/15)'] = vrr_trend['VRR']*0.85 + vrr_trend['VRR Trend - 50%']*0.15
vrr_trend['VRR+Trend-50% (80/20)'] = vrr_trend['VRR']*0.8 + vrr_trend['VRR Trend - 50%']*0.2
vrr_trend['VRR+Trend-50% (75/25)'] = vrr_trend['VRR']*0.75 + vrr_trend['VRR Trend - 50%']*0.25
vrr_trend['VRR+Trend-50% (70/30)'] = vrr_trend['VRR']*0.7 + vrr_trend['VRR Trend - 50%']*0.3
vrr_trend['VRR+Trend-25% (85/15)'] = vrr_trend['VRR']*0.85 + vrr_trend['VRR Trend - 25%']*0.15
vrr_trend['VRR+Trend-25% (80/20)'] = vrr_trend['VRR']*0.8 + vrr_trend['VRR Trend - 25%']*0.2
vrr_trend['VRR+Trend-25% (75/25)'] = vrr_trend['VRR']*0.75 + vrr_trend['VRR Trend - 25%']*0.25
vrr_trend['VRR+Trend-25% (70/30)'] = vrr_trend['VRR']*0.7 + vrr_trend['VRR Trend - 25%']*0.3
vrr_trend = vrr_trend[['VRR','VRR+Trend-50% (85/15)', 'VRR+Trend-50% (80/20)', 'VRR+Trend-50% (75/25)', 'VRR+Trend-50% (70/30)',
           'VRR+Trend-25% (85/15)', 'VRR+Trend-25% (80/20)', 'VRR+Trend-25% (75/25)', 'VRR+Trend-25% (70/30)']]
vrr_trend_dict = dm.get_data_dict(vrr_trend)
ret_data_dict_2 = dm.merge_dicts(ret_dict, vrr_trend_dict)

# Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict_2 = summary.get_corr_data(ret_data_dict_2, include_fi=include_fi)

# Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **corr** - Show:
    * **full** - full history correaltions
    * **equity_down** - risk seeking benchmark downside returns correlations or 
    * **equity_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict_2[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict_2 = summary.get_analytics_data(ret_data_dict_2,include_fi=True)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_2[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict_2 = summary.get_hist_data(ret_data_dict_2)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict_2[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **bmk** - Equity or FI bmk

In [ ]:
group_list=['Quintile','Decile']
bmk_list = list(ret_data_dict_2['Monthly'].select_dtypes('float').columns)[0:2]
@interact
def display_returns(group = group_list, bmk = bmk_list):
    quintile_df = summary.get_grouped_data(ret_data_dict_2, group = group, strat = bmk)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list,bmk = bmk_list, strat=list(ret_data_dict_2['Monthly'].select_dtypes('float').columns)[2:]):    
    df = summary.get_grouped_data(ret_data_dict_2, group = group, strat=bmk)
    
    for col in df.columns:
        if not (col == bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    

# Create Reports

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
if RUN_REPORTS:
    strat_report = 'vrr_plus_trend'
    selloffs = True
    rp.generate_strat_report(strat_report, ret_data_dict, selloffs)

# Equity Hedging Analysis
## Comparing with overal equity portfolio



## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [ ]:
equity_bmk = 'SPTR'
include_fi = False
strat_drop_list = ['Vortex','VRR','99%/90% Put Spread']
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Add VRR data

In [ ]:
returns = dm.merge_dicts(returns, vrr_dict)
for key in returns:
    returns[key] = returns[key][['SPTR', 'Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR','VRR Trend - 50%', 
                                 'VRR Trend - 25%','GW Dispersion','Corr Hedge', 'Def Var']]

## Create notional dict to compute different weighted hedges returns

In [ ]:
notional_dict = {
    'VRR':[19.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.25, 1.0],
    'VRR+Trend-50% (85/15)':[19.0, 1.0, 1.0, 1.0, 0.85, 0.15, 0.0, 1.0, 0.25, 1.0],
    'VRR+Trend-50% (80/20)':[19.0, 1.0, 1.0, 1.0, 0.8, 0.2, 0.0, 1.0, 0.25, 1.0],
    'VRR+Trend-50% (75/25)':[19.0, 1.0, 1.0, 1.0, 0.75, 0.25, 0.0, 1.0, 0.25, 1.0],
    'VRR+Trend-50% (70/30)':[19.0, 1.0, 1.0, 1.0, 0.7, 0.3, 0.0, 1.0, 0.25, 1.0],
    'VRR+Trend-25% (85/15)':[19.0, 1.0, 1.0, 1.0, 0.85, 0.0, 0.15, 1.0, 0.25, 1.0],
    'VRR+Trend-25% (80/20)':[19.0, 1.0, 1.0, 1.0, 0.8, 0.0, 0.2, 1.0, 0.25, 1.0],
    'VRR+Trend-25% (75/25)':[19.0, 1.0, 1.0, 1.0, 0.75, 0.0, 0.25, 1.0, 0.25, 1.0],
    'VRR+Trend-25% (70/30)':[19.0, 1.0, 1.0, 1.0, 0.7, 0.0, 0.3, 1.0, 0.25, 1.0],
    }

## Create Weighted Hedges Dict

In [ ]:
wh_ret = {}
for key  in returns:
    temp_df = returns[key].copy()
    temp_df = temp_df[[equity_bmk]]
    for notional in notional_dict:
        temp_wh = summary.util.get_weighted_hedges(returns[key], notional_dict[notional], include_fi,weight_col='WH_'+notional)
        temp_wh = temp_wh[['WH_'+notional]]
        temp_df = dm.merge_data_frames(temp_df, temp_wh)
    wh_ret[key] = temp_df

## View Returns data

Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [ ]:
freq_list = ['Daily', 'Weekly', 'Monthly','Quarterly', 'Yearly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(wh_ret[frequency])

## Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict_wh = summary.get_corr_data(wh_ret, corr_freq_list)

## Display Correlations

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict_wh[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and Hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict_wh = summary.get_analytics_data(wh_ret,analytics_freq_list)

## Display Analytics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict_wh[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(wh_ret)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile/Decile Analysis

In [ ]:
group_list=['Quintile','Decile']
@interact
def display_returns(group = group_list):
    quintile_df = summary.get_grouped_data(wh_ret, group = group)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile/Decile Analysis

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list, strat=list(wh_ret['Monthly'].select_dtypes('float').columns)[1:]):    
    df = summary.get_grouped_data(wh_ret, group = group)
    
    for col in df.columns:
        if not (col == equity_bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()

# Create Reports

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
if RUN_REPORTS:
    strat_report = 'weighted_hedges_vrr'
    selloffs = True
    rp.generate_strat_report(strat_report, wh_ret, selloffs)